## Number of operations

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

mnist_tmaxs = [2**exponent for exponent in range(1,6)]
mnist_error = 100 - np.array([89.68999981880188, 98.10999631881714, 99.04999732971191, 99.1599977016449, 99.24999475479126])
mnist_ops = np.array([49, 66, 101, 171, 310])*1e3

cifar_tmaxs = [2**exponent for exponent in range(3,7)]
cifar_error = 100 - np.array([83.30999612808228, 91.3599967956543, 92.14999675750732, 92.25999712944031])
cifar_ops = np.array([4.356, 6.912, 12.024, 22.249]) * 1e6

imagenet_tmaxs = [2**exponent for exponent in range(4,7)]
imagenet_error = 100 - np.array([54.44, 64.44, 67.24])
imagenet_ops = np.array([537.928, 775.855, 1251.711]) * 1e6
imagenet_times = [146, 290, 578]


df = pd.DataFrame({
    'Dataset': ['MNIST'] * 5 + ['CIFAR10'] * 4 + ['ImageNet'] * 3,
    'Text': mnist_tmaxs + cifar_tmaxs + imagenet_tmaxs,
    '% accuracy error': np.concatenate((mnist_error, cifar_error, imagenet_error)),
    'Ops per frame': np.concatenate((mnist_ops, cifar_ops, imagenet_ops)), 
})

df.loc[0, 'Text'] = "$T_{\mathrm{max}}=\mathrm{2}$"
df.loc[5, 'Text'] = "$T_{\mathrm{max}}=\mathrm{8}$"
df.loc[9, 'Text'] = "$T_{\mathrm{max}}=\mathrm{16}$"

fig = px.line(df, x="Ops per frame", y="% accuracy error", facet_col="Dataset", text="Text")
fig.update_traces(textposition=["middle right", "top right", "top right", "top center", "top center"], textfont=dict(size=11)) # "bottom left", "middle right"

ANN_kws = dict(showlegend=False, mode="markers+text", text=['<b>ANN</b>'], textfont=dict(size=14), textposition="top center", marker=dict(symbol='x', size=13, color='coral'))
fig.add_trace(go.Scatter(x=[0.22584e6], y=[100-99.22999739646912], **ANN_kws), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[171.680256e6], y=[100-92.38999485969543], **ANN_kws), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[7609.416192e6], y=[100-68.11000108718872], **ANN_kws), row = 1, col = 3)

fig.update_xaxes(type="log", matches=None, showticklabels=True)
fig.update_yaxes(matches=None, showticklabels=True)

import plotly.io as pio
pio.kaleido.scope.default_format = "svg"
pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 350

# fig['layout']['margin'] = dict(l=20, t=20)#['t'] = 0
# fig.show()
# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.write_image('n_ops.png')

# from IPython.display import Image
# img_bytes = fig.to_image(format="png", width=1000, height=350, scale=1)
# Image(img_bytes)

## Energy estimation

In [ ]:
energy_add = lambda b1, b2: max(b1, b2) + abs(b1-b2)/2
energy_mac = lambda b1, b2: b1*b2

mnist_energy = np.array([ops*energy_add(np.log2(tmax), 8) for ops, tmax in zip(mnist_ops, mnist_tmaxs)])
cifar_energy = np.array([ops*energy_add(np.log2(tmax), 8) for ops, tmax in zip(cifar_ops, cifar_tmaxs)])
imagenet_energy = np.array([ops*energy_add(np.log2(tmax), 8) for ops, tmax in zip(imagenet_ops, imagenet_tmaxs)])

df = pd.DataFrame({
    'Dataset': ['MNIST'] * 5 + ['CIFAR10'] * 4 + ['ImageNet'] * 3,
    'Text': mnist_tmaxs + cifar_tmaxs + imagenet_tmaxs,
    '% accuracy error': np.concatenate((mnist_error, cifar_error, imagenet_error)),
    'Relative dynamic energy per frame': np.concatenate((mnist_energy/0.22584e6/energy_mac(32,8), cifar_energy/171.680256e6/energy_mac(32,8), imagenet_energy/7609.416192e6/energy_mac(32,8))), 
})

df.loc[0, 'Text'] = "$T_{\mathrm{max}}=\mathrm{2}$"
df.loc[5, 'Text'] = "$T_{\mathrm{max}}=\mathrm{8}$"
df.loc[9, 'Text'] = "$T_{\mathrm{max}}=\mathrm{16}$"

fig = px.line(df, x="Relative dynamic energy per frame", y="% accuracy error", facet_col="Dataset", text="Text")
fig.update_traces(textposition=["middle right", "top right", "top right", "top right", "top right"], textfont=dict(size=11)) # "bottom left", "middle right"

ANN_kws = dict(showlegend=False, mode="markers+text", text=['<b>ANN 32bit</b>          '], textfont=dict(size=14), textposition="top center", marker=dict(symbol='x', size=13, color='coral'))
fig.add_trace(go.Scatter(x=[1], y=[100-99.22999739646912], **ANN_kws), row = 1, col = 1)
fig.add_trace(go.Scatter(x=[1], y=[100-92.38999485969543], **ANN_kws), row = 1, col = 2)
fig.add_trace(go.Scatter(x=[1], y=[100-68.11000108718872], **ANN_kws), row = 1, col = 3)

fig.update_xaxes(type="log", matches=None, showticklabels=True)
fig.update_yaxes(matches=None, showticklabels=True)

import plotly.io as pio
pio.kaleido.scope.default_format = "svg"
pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 350

fig.write_image('energy.png')
